 # LAB 1 Spark RDD#

## 1. RDD CREATION

### 1.1 Extract link from repository 


Given the repository "input1" which contains all the files, we build a list that contains all the files names. These latter will be useful to build our dataframe since we need the name of each City and the ID of each store.

In [72]:
mypath = 'input1/'
from os import listdir
from os.path import isfile, join
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
link = [mypath+f for f in files]
link

['input1/anger.txt',
 'input1/lyon.txt',
 'input1/marseilles_1.txt',
 'input1/marseilles_2.txt',
 'input1/nantes.txt',
 'input1/nice.txt',
 'input1/orlean.txt',
 'input1/paris_1.txt',
 'input1/paris_2.txt',
 'input1/paris_3.txt',
 'input1/rennes.txt',
 'input1/toulouse.txt',
 'input1/troyes.txt']

### 1.2 Extract City and ID from link

In this section, we made a function that takes a link to a file as input and return the correspending City and Store ID.
The store ID is the name of the city and the number of the store, if the city has only one store, we shall take City_1 as Store ID.

In [73]:
from pyspark.sql import Row

def extractStore(link):
  ## Function that compute the name of the city and its ID given the link 
  x = link.split('/')
  y = x[len(x)-1].split('.')[0] 
  z = y.split('_')
  if (len(z)>1):
    return x[len(x)-1].split('.')[0]
  else:
    City = y.split('_')[0]
    return City+'_1'


def extractCity(link):
  ## Function that compute the name of the city and its ID given the link 
  x = link.split('/')
  y = x[len(x)-1].split('.')[0] 
  z = y.split('_')
  return y.split('_')[0]

### 1.3 Create the data Frame 

To create the dataframe, we proceed by creating an RDD that contains the requested Rows and then, using an sqlContext, we create our dataframe passing the RDD as input.

To do so, we start by intializing four lists. The first one 

In [74]:

store = extractStore(link[0])
city = extractCity(link[0])
raw_data = sc.textFile(link[0])
data = raw_data.map(lambda line: line.split(" "))
rdd = data.map(lambda tokens: (store,tokens[0],tokens[1],city)).cache()

  
for i in range(1,13):
  store = extractStore(link[i])
  city = extractCity(link[i])

  raw_data = sc.textFile(link[i])
  data = raw_data.map(lambda line: line.split(" "))
  sm = data.map(lambda tokens: (store,tokens[0],tokens[1],city)).cache()
  rdd = rdd.union(sm)

rdd.collect()

[('anger_1', 'JAN', '13', 'anger'),
 ('anger_1', 'FEB', '12', 'anger'),
 ('anger_1', 'MAR', '14', 'anger'),
 ('anger_1', 'APR', '15', 'anger'),
 ('anger_1', 'MAY', '12', 'anger'),
 ('anger_1', 'JUN', '15', 'anger'),
 ('anger_1', 'JUL', '19', 'anger'),
 ('anger_1', 'AUG', '15', 'anger'),
 ('anger_1', 'SEP', '13', 'anger'),
 ('anger_1', 'OCT', '8', 'anger'),
 ('anger_1', 'NOV', '14', 'anger'),
 ('anger_1', 'DEC', '16', 'anger'),
 ('lyon_1', 'JAN', '13', 'lyon'),
 ('lyon_1', 'FEB', '12', 'lyon'),
 ('lyon_1', 'MAR', '14', 'lyon'),
 ('lyon_1', 'APR', '15', 'lyon'),
 ('lyon_1', 'MAY', '12', 'lyon'),
 ('lyon_1', 'JUN', '15', 'lyon'),
 ('lyon_1', 'JUL', '19', 'lyon'),
 ('lyon_1', 'AUG', '25', 'lyon'),
 ('lyon_1', 'SEP', '13', 'lyon'),
 ('lyon_1', 'OCT', '11', 'lyon'),
 ('lyon_1', 'NOV', '22', 'lyon'),
 ('lyon_1', 'DEC', '22', 'lyon'),
 ('marseilles_1', 'JAN', '21', 'marseilles'),
 ('marseilles_1', 'FEB', '21', 'marseilles'),
 ('marseilles_1', 'MAR', '21', 'marseilles'),
 ('marseilles_1', 'APR'

## 2. STATISTICS ON THE SHOP'S PERFORMANCE

### 2.1 Average monthly income of the shop in France

In [75]:
#Average monthly income of the shop in France
amsf = (rdd
        .map(lambda x : (x[1] , float(x[2])))
        .aggregateByKey((0,0), lambda a,b: (a[0] + b,a[1] + 1), lambda a,b: (a[0] + b[0], a[1] + b[1]))
        .mapValues(lambda x : (x[0] , float(x[1])))
        .mapValues(lambda x : x[0]/x[1] )
       )
        
amsf.collect()

[('MAR', 17.53846153846154),
 ('SEP', 25.53846153846154),
 ('JUN', 27.846153846153847),
 ('FEB', 19.153846153846153),
 ('OCT', 26.53846153846154),
 ('JUL', 21.692307692307693),
 ('MAY', 22.46153846153846),
 ('NOV', 24.53846153846154),
 ('AUG', 23.076923076923077),
 ('JAN', 20.76923076923077),
 ('DEC', 29.0),
 ('APR', 20.23076923076923)]

### 2.2 Average monthly income of the shop in each city

In [82]:
#Average monthly income of the shop in each city***
amc1 = (rdd
        .map(lambda x : [(x[1], x[3]) , float(x[2])])
        .aggregateByKey((0,0), lambda a,b: (a[0] + b,a[1] + 1), lambda a,b: (a[0] + b[0], a[1] + b[1]))
        .mapValues(lambda x : (x[0] / float(x[1])))
       )

amc1.sortByKey(False, 2).collect()

[(('SEP', 'troyes'), 21.0),
 (('SEP', 'toulouse'), 23.0),
 (('SEP', 'rennes'), 23.0),
 (('SEP', 'paris'), 48.0),
 (('SEP', 'orlean'), 13.0),
 (('SEP', 'nice'), 23.0),
 (('SEP', 'nantes'), 13.0),
 (('SEP', 'marseilles'), 23.0),
 (('SEP', 'lyon'), 13.0),
 (('SEP', 'anger'), 13.0),
 (('OCT', 'troyes'), 28.0),
 (('OCT', 'toulouse'), 14.0),
 (('OCT', 'rennes'), 18.0),
 (('OCT', 'paris'), 56.666666666666664),
 (('OCT', 'orlean'), 8.0),
 (('OCT', 'nice'), 18.0),
 (('OCT', 'nantes'), 14.0),
 (('OCT', 'marseilles'), 28.0),
 (('OCT', 'lyon'), 11.0),
 (('OCT', 'anger'), 8.0),
 (('NOV', 'troyes'), 11.0),
 (('NOV', 'toulouse'), 12.0),
 (('NOV', 'rennes'), 14.0),
 (('NOV', 'paris'), 48.666666666666664),
 (('NOV', 'orlean'), 24.0),
 (('NOV', 'nice'), 14.0),
 (('NOV', 'nantes'), 14.0),
 (('NOV', 'marseilles'), 24.0),
 (('NOV', 'lyon'), 22.0),
 (('NOV', 'anger'), 14.0),
 (('MAY', 'troyes'), 15.0),
 (('MAY', 'toulouse'), 11.0),
 (('MAY', 'rennes'), 11.0),
 (('MAY', 'paris'), 50.0),
 (('MAY', 'orlean'), 

### 2.3 Total revenue per city per year

In [77]:
#TOTAL REVENUE PER CITY PAR YEAR
trc = (rdd
       .map(lambda x : [x[3],int(x[2])])
       .reduceByKey(lambda x,y : (x+y))
       )
       
trc.collect()

[('paris', 1568),
 ('anger', 166),
 ('toulouse', 177),
 ('nice', 203),
 ('nantes', 207),
 ('lyon', 193),
 ('marseilles', 515),
 ('orlean', 196),
 ('troyes', 214),
 ('rennes', 180)]

### 2.4 Total revenue per store per year

In [78]:
#TOTAL REVENUE PER STORE PAR YEAR

trsy = (rdd
      .map(lambda x : [x[0],int(x[2])])
      .reduceByKey(lambda x,y : (x+y))
     )
     
trsy.collect()

[('rennes_1', 180),
 ('orlean_1', 196),
 ('paris_2', 642),
 ('lyon_1', 193),
 ('paris_3', 330),
 ('marseilles_2', 231),
 ('troyes_1', 214),
 ('nice_1', 203),
 ('toulouse_1', 177),
 ('marseilles_1', 284),
 ('anger_1', 166),
 ('paris_1', 596),
 ('nantes_1', 207)]

### 2.5 The store that achieves the best performance in each month

In [79]:
#BEST STORE PER MONTH
beststorepmonth = (rdd
                     .map(lambda x : [x[0],x[1],int(x[2])])
                     .map(lambda x: (x[1], x))
                     .reduceByKey(lambda x1, x2: max(x1, x2, key=lambda x: x[2])) 
                     .values()
                     .map(lambda x : [x[1],x[0]])
                   )
     
      
beststorepmonth.collect()

[['MAR', 'paris_2'],
 ['SEP', 'paris_2'],
 ['JUN', 'paris_2'],
 ['FEB', 'paris_2'],
 ['OCT', 'paris_1'],
 ['JUL', 'paris_1'],
 ['MAY', 'paris_2'],
 ['NOV', 'paris_2'],
 ['AUG', 'paris_2'],
 ['JAN', 'paris_1'],
 ['DEC', 'paris_1'],
 ['APR', 'paris_1']]